# Solid Segment Example

In [ ]:


import numpy as np
import time
import opensg.core.stress_recov as stress_recov
from opensg.mesh.segment import SolidSegmentMesh
from opensg.core.solid import compute_stiffness
import opensg

segment_stiffness_matrices = []
boundary_stiffness_matrices = []
compute_times = []
np.set_printoptions(precision=4)

out_file_name='bar_urc_npl_1_ar_5-segment_'
beam_out=opensg.utils.beamdyn_trans.beam_reaction(out_file_name)


# Process segments (adjust range based on desired segments)
for segment in np.linspace(1,1,1): # np.linspace(start segment, end segment, (end_segment-start segment+1))
    file_name='bar_urc_npl_1_ar_5-segment_'
    segid=int(segment)
    tic = time.time()
    print("\n Computing Segment:",str(segid)," \n")
    
    # Generate mesh for current segment using StandaloneSolidSegmentMesh
    mesh_yaml = file_name+ str(int(segment)) +'.yaml' 
    segment_mesh = SolidSegmentMesh(mesh_yaml)
    
    # The solid segment mesh provides the necessary data for stiffness computation
    
    # Extract material parameters and mesh data
    material_parameters, density = segment_mesh.material_database
    meshdata = segment_mesh.meshdata
    l_submesh = segment_mesh.left_submesh
    r_submesh = segment_mesh.right_submesh
    
    # Compute stiffness matrices using the solid analysis function
    timo = compute_stiffness(
        material_parameters,
        meshdata,
        l_submesh,
        r_submesh,
        boun=False)
    print(timo[0])
    
    file='solid.lp_sparcap_center_thickness_001.coords' 
    points=np.loadtxt(file, delimiter=' ', skiprows=0, dtype=str) # Load path coordinates
    points=[[float(pt[2]),float(pt[0]),float(pt[1])] for pt in points]
    # Dehomogenization
    strain_mm,_,u_loc=stress_recov.local_strain(timo,beam_out,segment,meshdata) # Local Strain
    
    stress_out=stress_recov.stress_eval(material_parameters, meshdata, strain_mm, points) # if points are given
    print(stress_out)
    # Eigen Solver
    eigen= opensg.core.stress_recov.eigen_solve(material_parameters,
                                    meshdata,
                                    l_submesh,
                                    r_submesh,
                                    strain_mm,
                                    u_loc)    
    
    
    
    
    
    
    
    
    